In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
import os

In [ ]:
import zipfile
import os

zip_path = "dataset.zip"  # or adjust if filename differs
extract_dir = "dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset unzipped to 'dataset/'")


✅ Dataset unzipped to 'dataset/'


In [ ]:
import os
import shutil
import pandas as pd

# Correct path to the CSV
csv_path = "dataset/dataset/Training_set.csv"
train_dir = "dataset/dataset/train"

# Load the CSV
df = pd.read_csv(csv_path)

# Normalize label formatting
def normalize(label):
    return label.strip().upper().replace(" ", "_")

# Organize files
for _, row in df.iterrows():
    filename = row['filename']
    label = normalize(row['label'])

    src = os.path.join(train_dir, filename)
    if not os.path.isfile(src):
        print(f"⚠️ File not found: {filename}")
        continue

    label_dir = os.path.join(train_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    dst = os.path.join(label_dir, filename)
    shutil.move(src, dst)
    print(f"✅ Moved {filename} → {label}/")

print("🎉 All images have been organized by class.")

Streaming output truncated to the last 5000 lines.
✅ Moved Image_1501.jpg → COPPER_TAIL/
✅ Moved Image_1502.jpg → DANAID_EGGFLY/
✅ Moved Image_1503.jpg → PAPER_KITE/
✅ Moved Image_1504.jpg → EASTERN_DAPPLE_WHITE/
✅ Moved Image_1505.jpg → PIPEVINE_SWALLOW/
✅ Moved Image_1506.jpg → PURPLE_HAIRSTREAK/
✅ Moved Image_1507.jpg → PIPEVINE_SWALLOW/
✅ Moved Image_1508.jpg → SCARCE_SWALLOW/
✅ Moved Image_1509.jpg → DANAID_EGGFLY/
✅ Moved Image_1510.jpg → GREY_HAIRSTREAK/
✅ Moved Image_1511.jpg → AMERICAN_SNOOT/
✅ Moved Image_1512.jpg → PIPEVINE_SWALLOW/
✅ Moved Image_1513.jpg → QUESTION_MARK/
✅ Moved Image_1514.jpg → AMERICAN_SNOOT/
✅ Moved Image_1515.jpg → BANDED_PEACOCK/
✅ Moved Image_1516.jpg → ELBOWED_PIERROT/
✅ Moved Image_1517.jpg → INDRA_SWALLOW/
✅ Moved Image_1518.jpg → BANDED_ORANGE_HELICONIAN/
✅ Moved Image_1519.jpg → ORCHARD_SWALLOW/
✅ Moved Image_1520.jpg → BECKERS_WHITE/
✅ Moved Image_1521.jpg → BECKERS_WHITE/
✅ Moved Image_1522.jpg → BANDED_PEACOCK/
✅ Moved Image_1523.jpg → AMERICA

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import json

# === Setup Image Generators ===
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    'dataset/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'dataset/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# === Load base model ===
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === Train ===
model.fit(train_gen, validation_data=val_gen, epochs=10)

# === Save model and class names ===
model.save('vgg16_model.h5')
with open('class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)

print("✅ Model and class_indices.json saved.")


Found 5225 images belonging to 75 classes.
Found 1274 images belonging to 75 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


164/164 ━━━━━━━━━━━━━━━━━━━━ 57s 274ms/step - accuracy: 0.0298 - loss: 4.3215 - val_accuracy: 0.1507 - val_loss: 3.9691
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 31s 191ms/step - accuracy: 0.1018 - loss: 3.8813 - val_accuracy: 0.2959 - val_loss: 3.4161
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 32s 197ms/step - accuracy: 0.2052 - loss: 3.3958 - val_accuracy: 0.4160 - val_loss: 3.0370
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 201ms/step - accuracy: 0.3006 - loss: 2.9523 - val_accuracy: 0.5047 - val_loss: 2.6578
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 199ms/step - accuracy: 0.3667 - loss: 2.6645 - val_accuracy: 0.5973 - val_loss: 2.3167
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 37s 224ms/step - accuracy: 0.4128 - loss: 2.4165 - val_accuracy: 0.6546 - val_loss: 2.1061
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 33s 201ms/step - accuracy: 0.4763 - loss: 2.1811 - val_accuracy: 0.6797 - val_loss: 1.9293
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 37s 224ms/step - accuracy: 0.5306 - loss: 1.9652 - val

✅ Model and class_indices.json saved.


In [ ]:
import shutil

# Create ZIP of the organized dataset
shutil.make_archive("organized_dataset", 'zip', "dataset")

# Download it
from google.colab import files
files.download("organized_dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("vgg16_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("class_indices.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>